# Experiments with Runnables

Create a few RunnableLambda, and compose them in sequence and in parallel

In [1]:
from langchain_core.runnables import (
    RunnablePassthrough,
    chain,
    RunnableParallel,
    RunnableLambda,
)
from devtools import debug
from asyncio import wait

add_1 = RunnableLambda(lambda x: x + 1)
add_3 = RunnableLambda(lambda x: x + 3)


@chain
def mult_2(x: int):
    return x * 2


sequence = mult_2 | add_1
parallel = mult_2 | {"add_1": add_1, "add_3": add_3}

# same as above:
parallel1 = RunnableParallel(add_1=add_1, add_3=add_3)

Run the runnable directly, batched, in parallel, ... 

In [2]:
sequence.invoke(1)  # 3
sequence.batch([1, 2, 3])  # [3,5,7]
parallel.invoke(1)  # {'add_1': 3, 'add_2': 4}

await sequence.abatch([1, 2, 3, 4, 5])

[3, 5, 7, 9, 11]

In [3]:
print("hello")

hello


In [ ]:
for s in sequence.stream(1000):
    print(s, end="", flush=True)

Print the graph and various type information

In [ ]:
parallel.get_graph().print_ascii()

print("input type:", sequence.InputType)
print("output type:", sequence.OutputType)

print("input schema: ", sequence.input_schema().schema())
print("output schema: ", sequence.output_schema().schema())

Use RunnablePassthrough

In [ ]:
runnable = RunnableParallel(origin=RunnablePassthrough(), modified=add_1)

runnable.invoke(1)  # {'origin': 1, 'modified': 2}

Demo 'bind' and 'RunnableConfig' : implement a filter, and log activities

In [ ]:
from langchain_core.runnables import RunnableConfig
from loguru import logger


@chain
def max(x: int, max: int, config: RunnableConfig) -> int:
    if log := (config["logger"]):  # type: ignore
        log.info(f"check if {x} < {max}")
    return max if x >= max else x


a = sequence | max.bind(max=6)
a.batch([1, 2, 3, 4, 5], {"logger": logger})

Demo 'assign", that adds new fields to the dict output of the runnable and returns a new runnable.

In [ ]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

runnable = (
    RunnableParallel(
        extra=RunnablePassthrough.assign(mult_10=lambda x: x["num"] * 10),
        plus_1=lambda x: x["num"] + 1,
    )
    .assign(info=lambda x: x)
    .assign(plus_1_time_3=lambda x: x["plus_1"] * 3)
)

runnable.invoke({"num": 2})

Most of the time, there are several parameters, or it's a dictionary (almost equivalent in Python).
'itemgetter' is used to get one field from a list of parameters.

In [ ]:
from operator import itemgetter


adder = RunnableLambda(lambda d: d["op1"] + d["op2"])


mult_2_and_add = (
    RunnableParallel(
        op1=RunnablePassthrough() | itemgetter("a") | mult_2,
        op2=RunnablePassthrough() | itemgetter("b") | mult_2,
    )
    | adder
)

mult_2_and_add.invoke({"a": 10, "b": 2})  # should return 2*10 + 2*2 = 24

In [ ]:
zzz = RunnableLambda(lambda x: {"a": x, "b": 2}) | mult_2_and_add

zzz.invoke(5)

We can easily add a fallback to any Runnable in case it fail

In [ ]:
@chain
def mult_2_fail(x: int):
    raise Exception("unavailable multiplication service")
    return x * 2


fallback_chain = mult_2_fail.with_fallbacks([mult_2])
fallback_chain.invoke(2)